In [72]:
    import pandas as pd
    import tensorflow_hub as hub
    import openai 
    import numpy as np
    import altair as alt
    import re
    from openai import OpenAI
    from sql_metadata import Parser
    from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
    from sklearn.decomposition import TruncatedSVD
    from sklearn.pipeline import make_pipeline
    from sklearn.preprocessing import Normalizer
    from sklearn.cluster import KMeans
    from sklearn.decomposition import PCA
    from sklearn.manifold import TSNE
    from sklearn.metrics import silhouette_score
    import spacy
    import nltk 
    from nltk.tokenize.toktok import ToktokTokenizer
    from bs4 import BeautifulSoup
    import unicodedata
    nltk.download('stopwords')
    nlp = spacy.load('en_core_web_lg')
    tokenizer = ToktokTokenizer()
    stopword_list = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cagatay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Introduction 
We want to identify the query patterns present in the Spider dataset. We are already familiar with some of the common SQL query patterns that exist, including **selection, aggregation, grouping, filtering, ordering, join, union, nested, insert, delete, update, and search**. 
We will perform an exploratory analysis on the dataset to gain a better understanding of its characteristics and identify any unique aspects. If you prefer, you can skip ahead to the [Results](#Results) section.

## Load the data

In [73]:
df = pd.read_parquet('data/train-00000-of-00001.parquet')

In [7]:
df

,db_id,query,question,query_toks,query_toks_no_value,question_toks
0,department_management,SELECT count(*) FROM head WHERE age > 56,How many heads of the departments are older th...,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...","[How, many, heads, of, the, departments, are, ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","List the name, born state and age of the heads...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","[List, the, name, ,, born, state, and, age, of..."
2,department_management,"SELECT creation , name , budget_in_billions ...","List the creation year, name and budget of eac...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","[List, the, creation, year, ,, name, and, budg..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...",What are the maximum and minimum budget of the...,"[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...","[What, are, the, maximum, and, minimum, budget..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,What is the average number of employees of the...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...","[What, is, the, average, number, of, employees..."
...,...,...,...,...,...,...
6995,culture_company,SELECT T1.company_name FROM culture_company AS...,What are all the company names that have a boo...,"[SELECT, T1.company_name, FROM, culture_compan...","[select, t1, ., company_name, from, culture_co...","[What, are, all, the, company, names, that, ha..."
6996,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...",Show the movie titles and book titles for all ...,"[SELECT, T1.title, ,, T3.book_title, FROM, mov...","[select, t1, ., title, ,, t3, ., book_title, f...","[Show, the, movie, titles, and, book, titles, ..."
6997,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...",What are the titles of movies and books corres...,"[SELECT, T1.title, ,, T3.book_title, FROM, mov...","[select, t1, ., title, ,, t3, ., book_title, f...","[What, are, the, titles, of, movies, and, book..."
6998,culture_company,SELECT T2.company_name FROM movie AS T1 JOIN c...,Show all company names with a movie directed i...,"[SELECT, T2.company_name, FROM, movie, AS, T1,...","[select, t2, ., company_name, from, movie, as,...","[Show, all, company, names, with, a, movie, di..."


Looking at the the _query_ column in the snippet, we don't see any `UPDATE`, `DELETE`, or `INSERT`queries. We want to verify if this is  the case for entire dataset. 

### Any `UPDATE/DELETE/INSERT` queries? 

In [74]:
df['query'].str.contains('UPDATE', re.I).sum()

0

In [75]:
df['query'].str.contains('DELETE', re.I).sum()

0

In [76]:
df['query'].str.contains('INSERT', re.I).sum()

0

So, there are no `UPDATE/DElETE/INSERT` queries.  **However, our check excluded any queries that might be calling a stored procedure (SP)**. Note that user defined functions (UDFs), the other kind of custom functions run in databases, can't perform data modification operations.  

In [6]:
df['query'].str.contains('EXEC', re.I).sum()

0

No SP calls. 

### Any indexing queries? 
Another pattern of queries is indexing queries but we don't see any in the samples. We want to also verify that. 

In [77]:
df['query'].str.contains('INDEX', re.I).sum()

0

There are no indexing queries either. 

### What about queries with constraints such as PRIMARY_KEY, FOREIGN_KEY, etc.?

In [78]:
df['query'].str.contains('PRIMARY_KEY|FOREIGN_KEY|UNIQUE|CREATE', re.I).sum()

0

Our analysis so far indicates that the dataset consists of read-only queries, which can be significant for optimization.

### <span style="color:green">Remark:</span> The dataset consists only of queries that are read-only. 

The sample queries suggest that we have various types of queries, such as selection, aggregation, grouping, filtering, ordering, join, union, and nested. We can count the queries of these types through pattern matching. Before doing that, it would be interesting to **explore if and how clustering can aid in discovering and visualizing these categories**. Let's proceed with that.


# Clustering SQL queries 
To cluster the queries, we'd like to first feuturize them, then select an appropriate number of clusters to pass it to the clustering algorithm. We'll use the [k-means](https://en.wikipedia.org/wiki/K-means_clustering) algorithm for clustering.


## Featurizing SQL queries 
When we look at the SQL queries in the dataset, they are all run against 170 different databases with the different schemas. 

### Generalizing SQL queries 
To capture general patterns with clustering, we need to standardize the database or schema specific aspects of queries as much as possible before featurizing them. 

For example, we'd want to transform a query such as `SELECT eid FROM Employee WHERE salary > 100000` to  `SELECT column_1 FROM table_1 WHERE column_2 > number_value`. We'll use the `generalize_query()` function defined below. 

Note the `generalize_query()` function uses `Parser()` from the [sql_metada](https://github.com/macbre/sql-metadata) package. We had to fix a bug in this package to make it work. 

In [79]:
# 
# Creates a generalized form of a query, providing a representation 
# independent of the database that it is executed on. 
# 
# The function relies on sql_metadata package, which is a bit 
# buggy and  incomplete, but nonetheless should provide a better 
# representation than a raw SQL query. 
# 
def generalize_query(query):
    q1 = Parser(query).generalize
    q2 = q1.replace(" TN ", " table ")
    q3 = q2.replace(" TN.", " table.")
    columns = Parser(query).columns
    tables = Parser(query).tables
    cnt = 1
    for t in tables: 
        q3 = q3.replace(t, "table_"+str(cnt))
        cnt += 1
    cnt = 1
    for c in columns:
        c = c.split('.')[-1]
        q3 = q3.replace('.'+c+' ', ".column_"+str(cnt)+' ')
        q3 = q3.replace('.'+c, ".column_"+str(cnt))
        q3 = q3.replace(' '+c+' ', " column_"+str(cnt)+' ')
        q3 = q3.replace('('+c+')', "(column_"+str(cnt)+')')
        cnt += 1
    q3 = q3.replace(' N', ' number_value')
    q3 = q3.replace(' N ', ' number_value ')
    q3 = q3.replace(' X', ' string_value')
    q3 = q3.replace(' X ', ' string_value ')
    
    return q3 
    

In [80]:
df['query_generalized'] = df['query'].apply(lambda x: generalize_query(x))

### Creating embedding vectors for generalized SQL queries

There are various methods for transforming text data into numerical features, such as the bag-of-words approach or embedding based on a language model. The success of these methods depends on the specific task and data. Since we don't have the time to perform a comprehensive analysis, we will use a transformer-based sentence encoder, the [**universal sentence encoder**](https://www.tensorflow.org/hub/tutorials/semantic_similarity_with_tf_hub_universal_encoder). Although OpenAI's text/code embedding models are also available, autoregressive models are not ideal for embedding text for similarity comparisons.

In [81]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4") 

In [82]:
query_embeddings = embed(df['query_generalized'])

2024-02-09 14:21:23.162118: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype string
	 [[{{node inputs}}]]


In [824]:
# 
# autoregressive transformer-based embedding 
#
# client = OpenAI(api_key='#')
# model='text-embedding-3-small'
# q = df['query_generalized'].to_list()
# e = []
# for r in range(0, 7000, 1000):
#     batch = q[r:r+1000]
#     batch_embedding = client.embeddings.create(input=batch, dimensions=512, model=model).data
#     batch_embedding = list(map(lambda x: x.embedding, batch_embedding))
#     e.extend(batch_embedding)

# query_embeddings = e

In [825]:
# bag of words 
# vectorizer = TfidfVectorizer(
#     max_df=0.5,
#     min_df=5,
#     stop_words="english",
# )
# query_embeddings = vectorizer.fit_transform(df['query_generalized'])
# lsa = make_pipeline(TruncatedSVD(n_components=20), Normalizer(copy=False))
# query_embeddings_pca = lsa.fit_transform(query_embeddings)

In [83]:
np.shape(query_embeddings)

TensorShape([7000, 512])

#### Using PCA to further reduce the dimensionality of the SQL vector embeddings 
We now have a 512-dimensional vector representation for each query. However, we will further reduce the dimensionality using [Principal Component Analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis) as our intuition and experience suggest that the degree of freedom between SQL queries is much lower than 512. In the following process, we go through a set of PCA components (similar to the elbow analysis) and choose **53** as the number of components we will use. This number suffices in explaining 90% of the variability in the data.

In [84]:
cum_var = []
pca_embeddings = []
num_components = range(2,65)
for n in num_components:
    m = PCA(n_components=n, random_state=1)
    pca_embeddings.append(m.fit_transform(query_embeddings))
    cum_var.append(m.explained_variance_ratio_.cumsum()[-1])


In [85]:
data = pd.DataFrame({
  'num_pca_components':list(num_components),
  'cum_explained_var': cum_var
})

alt.Chart(data).mark_line(point=True).encode(
    x='num_pca_components:O',
    y='cum_explained_var:Q',
    tooltip=['num_pca_components:O', 'cum_explained_var:Q']
).properties(
    width=800,
    height=200
)

alt.Chart(...)

In [86]:
query_embeddings_pca = PCA(n_components=53, random_state=1).fit_transform(query_embeddings)

## Selecting an appropriate number of clusters
We will use the vector embeddings in `query_embeddings_pca` to cluster the queries. However, when using the conventional k-means algorithm, we need to choose the number of clusters, or k value. To do this, we will evaluate different k values using the average [silhouette score](https://en.wikipedia.org/wiki/Silhouette_(clustering)), similarly to the PCA analysis previously conducted, and ultimately select **k = 13**. Note that these evaluations are imperfect tools used  to inform our decisions; the results from them change with underlying representations, parametric choices, random initializations, metrics used, and other factors. 

In [87]:
models = []
scores = []
nums_clusters = range(2, 21)
for n in nums_clusters:
    m = KMeans(n_clusters=n, n_init=10, init="k-means++", random_state=1)
    l = m.fit_predict(query_embeddings_pca)
    models.append((m,l))
    scores.append(silhouette_score(query_embeddings_pca, l))

In [88]:
data = pd.DataFrame({
  'num_clusters': list(nums_clusters),
  'avg_silhouette_score': scores
})

alt.Chart(data).mark_line(point=True).encode(
    x='num_clusters:N',
    y='avg_silhouette_score:Q',
    tooltip=['num_clusters:O', 'avg_silhouette_score:Q']
).properties(
    width=400,
    height=200
)

alt.Chart(...)

In [90]:
clustering_labels = KMeans(n_clusters=15, n_init=10, init="k-means++", random_state=1).fit_predict(query_embeddings_pca)

## Visualizing the clustering of SQL queries
To visualize the clustering results, we will create an interactive scatterplot (pan/zoom + tooltips) of the SQL queries. For each query, we will use [t-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding) to obtain the xy values and then color each circle representing a query in the plot by their cluster label obtained with the k-means clustering performed earlier.

In [91]:
query_embeddings_tsne = TSNE(n_components=2,random_state=1).fit_transform(query_embeddings_pca)

In [92]:
clustering_vis_data = pd.concat([pd.DataFrame(query_embeddings_tsne, columns=['x', 'y']), df['query'], df['question']], axis=1)

In [93]:
clustering_vis_data['cluster'] = clustering_labels

In [94]:
clustering_vis_data['id'] = np.arange(clustering_vis_data.shape[0])

In [95]:
clustering_vis_data['size'] = clustering_vis_data.groupby('cluster')['cluster'].transform('count')

In [96]:
clustering_vis_data['info'] = clustering_vis_data[['id', 'cluster', 'size']]\
.apply(lambda row: 'query_id:'+row[0].astype(str) + ', cluster_id:'+row[1].astype(str) + ', cluster_size:' + row[2].astype(str), axis=1)

/var/folders/qg/_d_3tv7j3ld15mr5jmgv7mxh0000gn/T/ipykernel_74325/355575894.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  .apply(lambda row: 'query_id:'+row[0].astype(str) + ', cluster_id:'+row[1].astype(str) + ', cluster_size:' + row[2].astype(str), axis=1)


In [31]:
alt.data_transformers.disable_max_rows()

alt.Chart(clustering_vis_data).mark_circle(size=80,opacity=0.5).encode(
    x='x:Q',
    y='y:Q',
    color=alt.Color('cluster:N').scale(scheme='category20'),
    tooltip=['info:N','query:N', 'question:N']
).properties(
    width=1000,
    height=1000
).interactive()

alt.Chart(...)

## Term frequencies of SQL clusters
In addition to the visualization above, it might be useful to look at the term frequencies ([TF-IDF scores](https://en.wikipedia.org/wiki/Tf–idf)) of SQL queris in each cluster. It can be considered another visualization of the clustering. 

In [99]:
def tfidf_scores(corpus, n=2):
  vectorizer = TfidfVectorizer(ngram_range = (n,n))
  X = vectorizer.fit_transform(corpus)
  features = vectorizer.get_feature_names_out()
  return X, features

In [100]:
def rank_features(X, features):
  sums = X.sum(axis = 0)
  data = []
  for col, term in enumerate(features):
    data.append( (term, sums[0,col] ))
  ranking = pd.DataFrame(data, columns = ['term','rank'])
  return (ranking.sort_values('rank', ascending = False))

In [101]:
def plot_by_rank(ranked_features, rank_type, title):
  W = 450
  H = 250
  chart=alt.Chart(ranked_features
  ).mark_bar().encode(
      x=alt.X('rank:Q', title=rank_type, axis=alt.Axis(titleFontSize=15, titleFontWeight=200, labelFont='Avenir', labelFontSize=12)),
      y=alt.Y('term:N', sort='-x', axis=alt.Axis(titleFontSize=15, labelFontSize=13,titleFontWeight=300, titleFont='Avenir'))
  ).properties(width=W,height=H, title=title)
  return chart

In [102]:
def plot_top_k(ranked_features, rank_type='frequency', k=10, title=None):
    title = title if title else 'Top ' + str(k) +' terms'
    return plot_by_rank(ranked_features.head(k), rank_type, title)

In [103]:
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [104]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [105]:
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    text = re.sub("'", "", expanded_text)
    return text

In [106]:
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text


In [107]:
def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

In [108]:
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    text = ' '.join(filtered_tokens)    
    return text

In [109]:
def normalize_corpus(corpus, contraction_expansion=True,
                     accented_char_removal=True, text_lower_case=True, 
                     text_lemmatization=True, special_char_removal=True, 
                     stopword_removal=True, remove_digits=True):
    
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:
        
        # remove accented characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)
        # expand contractions    
        if contraction_expansion:
            doc = expand_contractions(doc)
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        # lemmatize text
        if text_lemmatization:
            doc = lemmatize_text(doc)
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)  
        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)

        normalized_corpus.append(doc)
    
    return normalized_corpus 

In [112]:
def prep_tfidf_scores(df, labels, n=2, num_features=20):
    df_vis = pd.DataFrame(columns=['index', 'term', 'rank', 'cluster_id'])
    q = df['query_generalized']
    num_clusters = len(np.unique(labels))
    for i in range(num_clusters): 
        corpus = normalize_corpus(q[labels==i])
        X, features = tfidf_scores(corpus, n)
        ranked_features = rank_features(X, features)
        ranked_features = ranked_features[:num_features]
        ranked_features = ranked_features.reset_index()
        cluster_id = pd.DataFrame([i]*num_features, columns=['cluster_id'])
        cluster_tfidf = pd.concat([ranked_features, cluster_id], axis=1)
        df_vis = pd.concat([df_vis, cluster_tfidf], axis=0)
        
    return df_vis

In [114]:
df_ngram_vis = prep_tfidf_scores(df, clustering_labels)

/var/folders/qg/_d_3tv7j3ld15mr5jmgv7mxh0000gn/T/ipykernel_74325/469807748.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_vis = pd.concat([df_vis, cluster_tfidf], axis=0)


In [115]:
charts = []
for i in range(len(np.unique(clustering_labels))):
    data = df_ngram_vis [df_ngram_vis.cluster_id == i]
    chart = alt.Chart(data).mark_bar().encode(
        x=alt.X('rank:Q', title='tfidf score', axis=alt.Axis(titleFontSize=15, titleFontWeight=200, labelFont='Avenir', labelFontSize=12)),
        y=alt.Y('term:N', sort='-x', axis=alt.Axis(titleFontSize=15, labelFontSize=13,titleFontWeight=300, titleFont='Avenir'))
    ).properties(title='cluster # '+ str(i), width=256, height=256)
    charts.append(chart)

len(charts)
alt.hconcat(*charts[0:3]) & alt.hconcat(*charts[3:6]) & alt.hconcat(*charts[6:9])  & alt.hconcat(*charts[9:12]) & alt.hconcat(*charts[12:15])


alt.VConcatChart(...)

In [47]:
def count_queries(df, category):
    if category == 'selection': 
        m = df['query'].str.contains('SELECT', regex=True, flags=re.IGNORECASE)
    elif category == 'filtering': 
        m = df['query'].str.contains('^(?=.*(WHERE|HAVING)(?=.*(=|>|<|IN|BETWEEN|AND|OR|NOT|LIKE)))', flags=re.IGNORECASE, regex=True)
    elif category == 'aggregation': 
        m =  df['query'].str.contains('SUM|AVG|MAX|MIN', regex=True, flags=re.IGNORECASE)
    elif category == 'counting': 
        m = df['query'].str.contains('COUNT', flags=re.IGNORECASE, regex=True)
    elif category == 'grouping':
        m = df['query'].str.contains('GROUP BY', flags=re.IGNORECASE, regex=True)
    elif category == 'ordering': 
        m = df['query'].str.contains('ORDER BY',  flags=re.IGNORECASE, regex=True)
    elif category ==  'top-k': 
        m = m = df['query'].str.contains('^(?=.*ORDER BY )(?=.*LIMIT )', regex=True, flags=re.IGNORECASE)
    elif category == 'join':
        m = df['query'].str.contains('JOIN', regex=True, flags=re.IGNORECASE)
    elif category == 'intersect': 
        m = df['query'].str.contains('INTERSECT', regex=True, flags=re.IGNORECASE)
    elif category == 'union': 
        m = df['query'].str.contains('UNION', regex=True, flags=re.IGNORECASE)
    elif category == 'distinct': 
        m = df['query'].str.contains('DISTINCT', regex=True, flags=re.IGNORECASE)
    elif category == 'pattern_matching': 
        m = df['query'].str.contains('LIKE', regex=True, flags=re.IGNORECASE)
    elif category == 'nested':
        m = df['query'].str.contains('^(?=.*SELECT.*\(SELECT)', regex=True, flags=re.IGNORECASE)
    else: 
        print('Invalid category!')
        
    return (m.sum(),  df['cluster'][m].value_counts())
            
        

In [48]:
df['cluster'] = clustering_labels

In [49]:
query_category = ['selection', 'filtering', 'aggregation', 'counting','grouping', 
                  'ordering', 'top-k', 'join', 'intersect', 'union', 'distinct', 
                  'pattern_matching', 'nested']

query_counts = [count_queries(df, c)[0] for c in query_category] 

/var/folders/qg/_d_3tv7j3ld15mr5jmgv7mxh0000gn/T/ipykernel_74325/730340731.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  m = df['query'].str.contains('^(?=.*(WHERE|HAVING)(?=.*(=|>|<|IN|BETWEEN|AND|OR|NOT|LIKE)))', flags=re.IGNORECASE, regex=True)


# Results 

After our initial analysis by using clustering, text mining, visualizations, and textual pattern matching, we have identified **13**  categories of queries. These categories include **selection, filtering, aggregation, counting, grouping, ordering, top-k, join, intersect, union, distinct, pattern matching**, and **nested** queries. While these categories mostly overlap with general query patterns, the frequency of their usage differs from that of the queries in the wild. 

In [58]:
query_category_counts = pd.DataFrame({
  'query_category':query_category,
  'count': query_counts
})
                
alt.Chart(query_category_counts).mark_bar().encode(
        x=alt.X('count:Q', axis=alt.Axis(titleFontSize=15, titleFontWeight=200, labelFont='Avenir', labelFontSize=12)),
        y=alt.Y('query_category:N', sort='-x', axis=alt.Axis(title=None, 
                                                             titleFontSize=15, 
                                                             labelFontSize=13,
                                                             titleFontWeight=300, 
                                                             titleFont='Avenir')), 
        tooltip=['query_category:N', 'count:Q']
    ).properties(title='Approximate Frequency of Query Categories in Spider Dataset', width=500, height=300)

alt.Chart(...)


## Categories 

As we mentioned before, all the queries in the dataset are read-only. This might have implications for various query optimizations. 

Below, we briefly describe the 13 query categories within the context of the Spider SQL corpus.

* ### SELECTION queries

    - Queries that return (select) a subset of records from a database. 

    - All the queries in the query corpus are `SELECT` queries, as there are no "action" (delete, update, create, index, etc.) queries. 
    
    - However, there are only a few simple  `SELECT` queries (e.g., `SELECT column_1, column_2 FROM table`) that do not involve other SQL constructs beyond `FROM`. These simple queries are much less frequent, with only around 194 of them.
      
    - All the simple cases are clustered in Cluster 9.


* ### FILTERING queries
    
    - Queries that select records based on their match with criteria expressed in `WHERE` and `HAVING` clauses, involving, e.g., predicates 
    and operators such as `BETWEEN` and `IN`. 
    
    - FILTERING queries are the most common (~3875) queries in our corpus, comprising more than the half.  
    
    - They are concentrated in Clusters 6, 0, and 2 in the above visualizations.   
    
* ### AGGREGATION queries

    - Queries that calculate summary scalar values such as `SUM`, `AVG`, `MAX`, `MIN` over selections (we consider `COUNT` queries separately).
    
    - AGGREGATION queries are very common (~1112). 
    
    - They are typical data analytics queries and often repeated over identical tables and columns, making them ideal candidates for optimizations. 
    
    - In the above visualizations, AGGREGATION queries are distributed over clusters, while mostly concentrated in Clusters 0, 6, and 9.
    
* ### COUNTING queries 
    
    - Queries that calculate the count of records over matching selections. 
    
    - `COUNT` queries  are the third most common category in our corpus with around 2705 instances. 
    
    - These queries, similar to other aggregation queries, are spread throughout all clusters, but are more concentrated in Clusters 2, 10, and 1. 
    
    - Optimizations based on data sketching can improve the performance of these queries.
     
     
* ### GROUPING queries
    
    - Queries that group data based on  column values using `GROUP BY` clause. `GROUP BY` is often used before aggregation operations. 
    
    - In the dataset, `WHERE` clauses come before `GROUP BY`,  while `HAVING` clauses come after, which is in line in common practices. 
    
    - GROUPING queries are the fourth most frequent (~1775) queries in the corpus and often used together with `COUNT`. 
    
    - There are concentrated in Clusters 2, 12, 10, and 1. 
    

* ### ORDERING queries

    - Queries that sort records by a specifict column using ORDER BY, frequently accompanied by `DESC` (`ORDER BY` sorts in ascending order by default)
    
    - In the corpus, ORDERING queries are common (~1628). 
    
    - Clusters 1, 14, and 10 contain most of the ORDERING queries. 
    
* ### TOP-k queries

    - An important subclass of ORDERING queries in this dataset is TOP-k queries, where k is specified by `LIMIT` and is typically 1. 
    
    - Most (~1107) of the ORDERING queries in the dataset are TOP-k queries.
    
    - TOP-k queries can be expensive, particularly when k = 1 (single point queries), which is the most frequent (~1001) case, and should be targeted for optimizations.
    
    - Most of the TOP-k queries are in Clusters 1, 10, and 3 in the above visaulizations. 


* ### `JOIN` queries
    
    - Queries that combine (join) columns from multiple tables (e.g. _Show the name of all bridges that was designed by an American architect, and sort the result by the bridge feet length_ --> `SELECT t1.name FROM bridge AS t1 JOIN architect AS t2 ON t1.architect_id  =  t2.id WHERE t2.nationality  =  'American' ORDER BY t1.length_feet`)
    
    - `JOIN` queries are the second most common (~2783) queries in the dataset. 
    
    - In the clustering visualizations above, `JOIN` queries are primarily represented in Clusters 6, 2, and 1.  
    
    - There is also a significant number (~827) of **multi-`JOIN`** queries. These queries typically represent complex operations that could benefit from optimizations.
    
    - Clusters 6, 5, and 2 contain most of the multi-`JOIN` queries.
    

* ### `INTERSECT` queries 

    - Queries that intersect records from two select statements, using the `INTERSECT` clause. 
    
    - While more frequent than `UNION` queries, compared to `JOIN` queries, `INTERSECT` queries are less frequent (~250) in the query corpus. 
    
    - Most of these queries are in Clusters 6, 0, and 8. 
    
    
* ### `UNION` queries 

    - Queries that combine (stack on top of each other) information from two select statements using the `UNION` clause. 
    
    - Unlike JOIN queries, UNION queries are infrequent (~67) in the  corpus. They are the least frequent queries in the corpus. 
    
    - Most of these queries are in Clusters 0, 6, and 8. 


* ### `DISTINCT` queries
    
    - Queries retrieves unique values from specified columns using DISTINCT.
    
    - `DISTINCT` queries occur occassionally (~721). 
    
    - They are primarily grouped in Clusters 6, 11, and 9. 
    

* ### PATTERN MATCHING queries 

    - Queries that match character patterns in column values using the `LIKE` operator (e.g., _Find all the catalog publishers whose name contains "Murray"_ -->`SELECT distinct(catalog_publisher) FROM catalogs WHERE catalog_publisher LIKE "%Murray%"`) 

    - PATTERN MATCHING queries are infrequent (~171) in our corpus, being the second least frequent. 
    
    - Nevertheless, these queries can be expensive and, hence, important for performance optimization. 
    
    - Almost all the PATTERN MATCHING queries are concentrated in Clusters 0, 6, and 13. 
    

* ### NESTED queries 
    
    - Queries involving one or more subqueries (e.g., _List the id of students who never attends courses_ -->`SELECT * FROM student_course_registrations WHERE student_id NOT IN (SELECT student_id FROM student_course_attendance`)
    
    - The corpus has a modest size (~452) of NESTED queries. 
    
    - Crucially, these queries are often complex queries and can be a source of performance bottlenecks.  
    
    - Most of these queries are grouped in Clusters 8, 0, and 6 in the visualizations above.
    